In [41]:
import requests
import pandas as pd
import datetime
import time
import os 


In [ ]:
# ========== CONFIG — EDIT THESE ==========
symbols = pd.read_excel(r"C:\Users\A\Desktop\algo_project\symbol_key.xlsx")
nifty_folder = os.path.join(os.getcwd(), "nifty_symbols")

ACCESS_TOKEN =  "" # Paste your Upstox OAuth2 access token here
# INSTRUMENT_KEY = "NSE_INDEX|Nifty Bank"  # Bank Nifty index key. Change if you want BANKNIFTY futures.

In [14]:
instrument_keys = dict(zip(symbols['Symbol'], symbols['Key']))


In [26]:
# Date range for data
START_DATE = "2022-01-01"  # YYYY-MM-DD
END_DATE   = "2025-11-30"  # YYYY-MM-DD

In [45]:
# Throttle between API calls (in seconds) to avoid rate limits
SLEEP_AFTER_ONE_PULL = 1
SLEEP_WITHIN_SYMBOLS = 5

========== END CONFIG ==========

In [46]:
# Base URL for Upstox V3 Historical Candle API
BASE_URL = "https://api.upstox.com/v3/historical-candle"

In [47]:
def fetch_candles_month(instrument_key, unit, interval, from_date, to_date, token):
    """
    Fetch candle data for a month-range using Upstox V3 API.
    unit: "minutes" / "hours" / "days" etc.
    interval: integer as string: "15" for 15-min if unit is minutes.
    from_date, to_date: "YYYY-MM-DD" (inclusive)
    """
    url = f"{BASE_URL}/{instrument_key}/{unit}/{interval}/{to_date}/{from_date}"
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/json",
    }
    resp = requests.get(url, headers=headers)
    resp.raise_for_status()
    return resp.json()

In [48]:
def month_ranges(start: datetime.date, end: datetime.date):
    """Generator that yields (month_start_date, month_end_date) pairs."""
    cur = start
    while cur < end:
        # Next month
        next_month = (cur.replace(day=1) + datetime.timedelta(days=32)).replace(day=1)
        # But cap at end date
        month_end = min(next_month - datetime.timedelta(days=1), end)
        yield cur, month_end
        cur = next_month

In [51]:
def main():
    # Parse dates
    start_dt = datetime.datetime.strptime(START_DATE, "%Y-%m-%d").date()
    end_dt = datetime.datetime.strptime(END_DATE, "%Y-%m-%d").date()
    for symbol, key in keys.items():
        print(f"\n=== Processing {symbol} ===")
        all_candles = []

        for month_start, month_end in month_ranges(start_dt, end_dt):
            print(f"Fetching: {month_start} → {month_end} ...")
            try:
                result = fetch_candles_month(
                    key.replace("'",""),
                    unit="minutes",
                    interval="1",
                    from_date=month_start.isoformat(),
                    to_date=month_end.isoformat(),
                    token=ACCESS_TOKEN,
                )
            except Exception as e:
                print("Error fetching month:", e)
                break

            # Example response schema (from docs): result["data"]["candles"]
            candles = result.get("data", {}).get("candles", [])
            if not candles:
                print("No candles in this range (or empty).")
            for c in candles:
                ts = c[0]

                # Handle timestamp format
                if isinstance(ts, str):
                    # API returns like "2022-01-03T09:15:00+05:30"
                    timestamp = datetime.datetime.fromisoformat(ts.replace("Z", "+00:00"))
                else:
                    timestamp = datetime.datetime.fromtimestamp(ts / 1000)

                all_candles.append({
                    "timestamp": timestamp,
                    "open": c[1],
                    "high": c[2],
                    "low": c[3],
                    "close": c[4],
                    "volume": c[5],
                    "open_interest": c[6] if len(c) > 6 else None,
                })
            # time.sleep(SLEEP_AFTER_ONE_PULL)
        # Convert to DataFrame
        df = pd.DataFrame(all_candles)
        df.sort_values("timestamp", inplace=True)
        df["timestamp"] = df["timestamp"].dt.tz_localize(None)
        df.set_index("timestamp", inplace=True)
        # Save to CSV and Excel
        output_csv =os.path.join(nifty_folder,f"{symbol}.csv")
        df.to_csv(output_csv, index="True")
        print("Done! Saved", len(df), "candles.")
        print("→", output_csv)
        time.sleep(SLEEP_WITHIN_SYMBOLS)

In [52]:
if __name__ == "__main__":
    main()


=== Processing TATASTEEL ===
Fetching: 2022-01-01 → 2022-01-31 ...
Fetching: 2022-02-01 → 2022-02-28 ...
Fetching: 2022-03-01 → 2022-03-31 ...
Fetching: 2022-04-01 → 2022-04-30 ...
Fetching: 2022-05-01 → 2022-05-31 ...
Fetching: 2022-06-01 → 2022-06-30 ...
Fetching: 2022-07-01 → 2022-07-31 ...
Fetching: 2022-08-01 → 2022-08-31 ...
Fetching: 2022-09-01 → 2022-09-30 ...
Fetching: 2022-10-01 → 2022-10-31 ...
Fetching: 2022-11-01 → 2022-11-30 ...
Fetching: 2022-12-01 → 2022-12-31 ...
Fetching: 2023-01-01 → 2023-01-31 ...
Fetching: 2023-02-01 → 2023-02-28 ...
Fetching: 2023-03-01 → 2023-03-31 ...
Fetching: 2023-04-01 → 2023-04-30 ...
Fetching: 2023-05-01 → 2023-05-31 ...
Fetching: 2023-06-01 → 2023-06-30 ...
Fetching: 2023-07-01 → 2023-07-31 ...
Fetching: 2023-08-01 → 2023-08-31 ...
Fetching: 2023-09-01 → 2023-09-30 ...
Fetching: 2023-10-01 → 2023-10-31 ...
Fetching: 2023-11-01 → 2023-11-30 ...
Fetching: 2023-12-01 → 2023-12-31 ...
Fetching: 2024-01-01 → 2024-01-31 ...
Fetching: 2024-02-01